<a href="https://colab.research.google.com/github/KoreTane/PredictiveLeadManagement/blob/main/Forecasting_marketing_investments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

In [2]:
dfm = pd.read_excel('MatriculaCEP.xlsx')
dfc = pd.read_excel('fluxo_caixa_cbs.xlsx')

In [3]:
dfc['Matricula'] = dfc['Descrição'].str.split('#', expand=True)[2]
dfc['Aluno'] = dfc['Descrição'].str.replace(r'#\d+|[0-9]', '', regex=True)
dfc = dfc['Data', 'Matricula', 'Aluno',  ]

In [5]:
dfc


,Data,Descrição,Valor,Tipo,Matricula
0,2023-01-11,#CLIENTE,244.02,Débito,23017
1,2023-01-12,#CLIENTE,199.00,Dinheiro,23018
2,2023-01-13,#CLIENTE,237.12,Crédito,23002
3,2023-01-13,#CLIENTE,237.12,Crédito,23003
4,2023-01-14,#CLIENTE,250.00,Dinheiro,23007
...,...,...,...,...,...
1306,2024-04-15,#CLIENTE,230.00,Dinheiro,24024
1307,2024-04-17,#CLIENTE,164.07,Crédito,24024
1308,2024-04-23,#CLIENTE,316.89,Crédito,24023
1309,2024-04-23,#CLIENTE,358.34,Crédito,24023
